# Google Drive Transfer Tool with rclone

This notebook provides a simple graphical interface for transferring files between Google Drive remotes using rclone. Perfect for migrating files between different Google accounts or organizing your data.

## 📋 Prerequisites
- You need an `rclone.conf` file with your Google Drive configurations
- If you don't have one, see the [rclone documentation](https://rclone.org/drive/) for setup instructions

## 🚀 Getting Started
1. Run the setup cell below
2. Upload your rclone.conf file
3. Configure your transfer settings
4. Start the transfer

---

## Setup: Install Dependencies

Run this cell first to install rclone and required Python libraries.

In [ ]:
# Install rclone and dependencies
import subprocess
import sys
import os
from pathlib import Path

print("🔧 Installing dependencies...")

# Install rclone
try:
    result = subprocess.run(['which', 'rclone'], capture_output=True, text=True)
    if result.returncode != 0:
        print("📦 Installing rclone...")
        subprocess.run(['curl', 'https://rclone.org/install.sh'], stdout=subprocess.PIPE, check=True)
        subprocess.run(['bash'], input='curl https://rclone.org/install.sh | sudo bash', shell=True, check=True)
    else:
        print("✅ rclone already installed")
except Exception as e:
    print(f"⚠️  Installing rclone manually...")
    !curl https://rclone.org/install.sh | sudo bash

# Install Python packages
print("📦 Installing Python packages...")
!pip install -q ipywidgets

# Enable ipywidgets for Colab
try:
    from google.colab import output
    output.enable_custom_widget_manager()
    print("✅ ipywidgets enabled for Colab")
except ImportError:
    print("ℹ️  Not running in Colab, widgets should work normally")

# Create rclone config directory
config_dir = Path('/root/.config/rclone')
config_dir.mkdir(parents=True, exist_ok=True)

print("\n✅ Setup complete! You can now proceed to the next steps.")

## Step 1: Upload rclone.conf

Upload your rclone configuration file containing your Google Drive remote configurations.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import io
import os

# Global variable to track config upload status
config_uploaded = False

# File upload widget
upload_widget = widgets.FileUpload(
    accept='.conf',
    multiple=False,
    description='Choose rclone.conf',
    style={'description_width': 'initial'}
)

# Status output
status_output = widgets.Output()

def handle_upload(change):
    global config_uploaded
    
    with status_output:
        clear_output(wait=True)
        
        if upload_widget.value:
            try:
                # Get the uploaded file
                filename, file_info = next(iter(upload_widget.value.items()))
                content = file_info['content']
                
                # Save to rclone config location
                config_path = '/root/.config/rclone/rclone.conf'
                with open(config_path, 'wb') as f:
                    f.write(content)
                
                # Verify the file was saved
                if os.path.exists(config_path):
                    config_uploaded = True
                    print("✅ rclone.conf uploaded successfully!")
                    print(f"📁 Saved to: {config_path}")
                    
                    # Show available remotes
                    try:
                        import subprocess
                        result = subprocess.run(['rclone', 'listremotes'], 
                                               capture_output=True, text=True, check=True)
                        if result.stdout.strip():
                            print("\n📋 Available remotes:")
                            for remote in result.stdout.strip().split('\n'):
                                if remote.strip():
                                    print(f"  • {remote}")
                        else:
                            print("⚠️  No remotes found in configuration file")
                    except subprocess.CalledProcessError:
                        print("⚠️  Could not list remotes. Please check your configuration.")
                else:
                    print("❌ Failed to save configuration file")
                    config_uploaded = False
                    
            except Exception as e:
                print(f"❌ Error uploading file: {str(e)}")
                config_uploaded = False

# Bind the upload handler
upload_widget.observe(handle_upload, names='value')

# Display widgets
print("📤 Select your rclone.conf file:")
display(upload_widget)
display(status_output)

## Step 2: Configure Your Transfer

Set up your source and destination paths, transfer method, and rclone options.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML

# Source and Destination inputs
source_input = widgets.Text(
    placeholder='source_drive:path/to/folder',
    description='Source Path:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='500px')
)

destination_input = widgets.Text(
    placeholder='destination_drive:path/to/folder',
    description='Destination Path:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='500px')
)

# Transfer Method
transfer_mode = widgets.RadioButtons(
    options=[
        ('Server-Side (Recommended) - Fast, direct transfer between drives', 'server-side'),
        ('Local Colab Transfer - Slower, routes through Colab VM', 'local')
    ],
    value='server-side',
    description='Transfer Mode:',
    style={'description_width': 'initial'}
)

# rclone Options
dry_run_checkbox = widgets.Checkbox(
    value=False,
    description='--dry-run: Test the transfer without copying files',
    style={'description_width': 'initial'}
)

fast_list_checkbox = widgets.Checkbox(
    value=True,
    description='--fast-list: Use less memory and time for directory listings',
    style={'description_width': 'initial'}
)

verbose_checkbox = widgets.Checkbox(
    value=False,
    description='--verbose: Show verbose logging',
    style={'description_width': 'initial'}
)

# Parallel transfers slider
transfers_slider = widgets.IntSlider(
    value=4,
    min=1,
    max=32,
    step=1,
    description='Transfers:',
    style={'description_width': '100px'}
)

transfers_label = widgets.Label('Number of parallel file transfers')

# Checkers slider
checkers_slider = widgets.IntSlider(
    value=8,
    min=1,
    max=64,
    step=1,
    description='Checkers:',
    style={'description_width': '100px'}
)

checkers_label = widgets.Label('Number of parallel checkers')

# Layout the widgets
print("🔧 Configure your transfer settings:")
print()

display(HTML("<h4>📂 Paths</h4>"))
display(source_input)
display(destination_input)

display(HTML("<h4>⚡ Transfer Method</h4>"))
display(transfer_mode)

display(HTML("<h4>⚙️ rclone Options</h4>"))
display(dry_run_checkbox)
display(fast_list_checkbox)
display(verbose_checkbox)

display(HTML("<h4>🔄 Performance Settings</h4>"))
display(widgets.HBox([transfers_slider, transfers_label]))
display(widgets.HBox([checkers_slider, checkers_label]))

## Step 3: Run Transfer

Execute the transfer with your configured settings.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import subprocess
import threading
import time
import os

# Start transfer button
start_button = widgets.Button(
    description='▶️ Start Transfer',
    button_style='success',
    layout=widgets.Layout(width='200px', height='50px')
)

# Stop button (initially disabled)
stop_button = widgets.Button(
    description='⏹️ Stop Transfer',
    button_style='danger',
    layout=widgets.Layout(width='200px', height='50px'),
    disabled=True
)

# Output area for command and results
output_area = widgets.Output()

# Global variable to track the running process
current_process = None

def build_rclone_command():
    """Build the rclone command from user inputs"""
    cmd = ['rclone', 'copy']
    
    # Add source and destination
    cmd.extend([source_input.value.strip(), destination_input.value.strip()])
    
    # Add transfer mode specific flags
    if transfer_mode.value == 'server-side':
        cmd.append('--drive-server-side-across-configs=true')
    
    # Add optional flags
    if dry_run_checkbox.value:
        cmd.append('--dry-run')
    
    if fast_list_checkbox.value:
        cmd.append('--fast-list')
    
    if verbose_checkbox.value:
        cmd.append('--verbose')
    
    # Add performance settings
    cmd.append(f'--transfers={transfers_slider.value}')
    cmd.append(f'--checkers={checkers_slider.value}')
    
    # Add progress and stats
    cmd.extend(['--progress', '--stats=10s'])
    
    return cmd

def validate_inputs():
    """Validate user inputs before starting transfer"""
    errors = []
    
    # Check if config is uploaded
    if not config_uploaded or not os.path.exists('/root/.config/rclone/rclone.conf'):
        errors.append("❌ Please upload your rclone.conf file first")
    
    # Check source path
    if not source_input.value.strip():
        errors.append("❌ Source path is required")
    elif ':' not in source_input.value:
        errors.append("❌ Source path should include remote name (e.g., 'remote:path')")
    
    # Check destination path
    if not destination_input.value.strip():
        errors.append("❌ Destination path is required")
    elif ':' not in destination_input.value:
        errors.append("❌ Destination path should include remote name (e.g., 'remote:path')")
    
    return errors

def run_transfer_thread(cmd):
    """Run the transfer in a separate thread"""
    global current_process
    
    try:
        with output_area:
            print("🚀 Starting transfer...\n")
            
            # Start the process
            current_process = subprocess.Popen(
                cmd,
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                universal_newlines=True,
                bufsize=1
            )
            
            # Read output line by line
            for line in current_process.stdout:
                print(line.rstrip())
            
            # Wait for completion and get return code
            return_code = current_process.wait()
            
            if return_code == 0:
                print("\n✅ Transfer completed successfully!")
            else:
                print(f"\n❌ Transfer failed with exit code: {return_code}")
                
    except Exception as e:
        with output_area:
            print(f"\n❌ Error during transfer: {str(e)}")
    
    finally:
        # Re-enable start button and disable stop button
        start_button.disabled = False
        stop_button.disabled = True
        current_process = None

def on_start_click(b):
    """Handle start button click"""
    global current_process
    
    with output_area:
        clear_output(wait=True)
        
        # Validate inputs
        errors = validate_inputs()
        if errors:
            for error in errors:
                print(error)
            return
        
        # Build command
        cmd = build_rclone_command()
        
        # Display the command that will be executed
        print("📋 Command to be executed:")
        print(f"   {' '.join(cmd)}")
        print("\n" + "="*60 + "\n")
        
        # Update button states
        start_button.disabled = True
        stop_button.disabled = False
        
        # Start transfer in a separate thread
        transfer_thread = threading.Thread(target=run_transfer_thread, args=(cmd,))
        transfer_thread.daemon = True
        transfer_thread.start()

def on_stop_click(b):
    """Handle stop button click"""
    global current_process
    
    if current_process:
        with output_area:
            print("\n⏹️ Stopping transfer...")
        
        try:
            current_process.terminate()
            time.sleep(2)
            if current_process.poll() is None:
                current_process.kill()
        except:
            pass
        
        start_button.disabled = False
        stop_button.disabled = True
        current_process = None
        
        with output_area:
            print("❌ Transfer stopped by user")

# Bind button handlers
start_button.on_click(on_start_click)
stop_button.on_click(on_stop_click)

# Display the interface
print("🎯 Ready to transfer! Click the button below to start:")
display(widgets.HBox([start_button, stop_button]))
display(output_area)

---

## 📚 Additional Information

### Transfer Modes Explained:
- **Server-Side Transfer**: Files are copied directly between Google Drive accounts using Google's servers. This is much faster and doesn't use Colab's bandwidth or storage.
- **Local Colab Transfer**: Files are downloaded to the Colab VM and then uploaded to the destination. This is slower and uses Colab resources.

### Common rclone.conf Setup:
If you don't have an rclone.conf file:
1. Install rclone locally on your computer
2. Run `rclone config` to set up your Google Drive remotes
3. Find the config file (usually at `~/.config/rclone/rclone.conf` on Linux/Mac or `%APPDATA%\rclone\rclone.conf` on Windows)
4. Upload it using the file upload widget above

### Tips:
- Use `--dry-run` first to test your transfer without actually copying files
- Server-side transfers work best between Google Drive remotes
- Adjust the number of transfers and checkers based on your needs (more = faster but more resource intensive)

### Troubleshooting:
- If the transfer fails, check that your remote names in the paths match those in your rclone.conf
- Make sure your rclone.conf has valid authentication tokens
- For large transfers, consider running multiple smaller transfers

---
*Made with ❤️ for easy Google Drive transfers*